In [2]:
%matplotlib inline

from osgeo import gdal, ogr, osr
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import matplotlib.pyplot as plt
import numpy as np
import os
import cenpy as cen

C:\Users\zstat\Anaconda2\envs\py36\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://migrating.pysal.org
  ), VisibleDeprecationWarning)


## Explore the National Land Cover Database (NLCD) raster

In [ ]:
NLCD = '../data/NLCD/NLCD2011_LC_New_Jersey/NLCD2011_LC_New_Jersey.tif'

In [ ]:
# Check coordinate system and exclude values outside of the land cover classification numbers
with rasterio.open(NLCD) as nlcd_raster:
    crs = nlcd_raster.crs
    nlcd = nlcd_raster.read(1)
    #nlcd = nlcd.astype('f4')
    # Set cells to 0
    nlcd[nlcd>95] = 0

crs
nlcd.dtype

In [ ]:
#Identify the count of each value in the raster
(values, counts) = np.unique(nlcd, return_counts=True)
values = dict(zip(values, counts))
values

In [ ]:
a=np.hstack(nlcd)
plt.hist(a, bins='auto')
plt.show

In [ ]:
# Map array
plt.imshow(nlcd)
plt.colorbar()
plt.xlabel('Column #')
plt.ylabel('Row #')


## Identify only residential land

In [ ]:
# Create a function to only identify residential land. These values are provided in the metadata about the NLCD.
res_values =  [22, 23, 24]
non_res_values = [11,12,21,31,41,42,43,51,52,71,72,81,82,90,95]

def identify_res_land(array):
    for val in res_values:
        array[array==val] = 1
    for val in non_res_values:
        array[array==val] = 0
        
    return array

In [ ]:
# Map new array
nj_res_land = identify_res_land(nlcd)

plt.imshow(nj_res_land)
leg = plt.colorbar()
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
# Preparing to export new array as a raster (GeoTIFF)
out_fp = '../data/Created/NJ_residential.tif'

In [ ]:
# If the file exists, delete it
if os.path.exists(out_fp):
    os.remove(out_fp)
else:
    print("Cannot delete the file as it doesn't exists")

In [ ]:
# Get original raster's coordinate system and affine transformation
with rasterio.open(NLCD) as source_raster:
    source_crs = source_raster.crs
    source_transform = source_raster.transform
print(source_crs)
print(source_transform)

In [ ]:
# Export array to raster(GeoTiff)
with rasterio.open(out_fp, 'w', driver='GTiff',
                   height=nj_res_land.shape[0], # Number of rows
                   width=nj_res_land.shape[1], # Number of columns
                   count=1, # Number of bands
                   dtype=nj_res_land.dtype, # This must match the datatpe of the array
                   crs=source_crs,
                   transform=source_transform
                  ) as out_raster:
    out_raster.write(nj_res_land, 1)
    out_raster.nodata = -1 # Set the raster's nodata value

In [ ]:
# Map and examine new raster
exported = rasterio.open(out_fp)
rasterio.plot.show(exported)

## Crop raster for experimentation

Since I was coming across some problems converting the raster into polygons and gdal.Polygonize() was quite slow, I clipped the raster in order to experiment on a smaller dataset.

In [ ]:
original = '../data/Created/NJ_residential.tif'
clip = '../data/Created/NJ_res_clip.tif'
data = rasterio.open(original)

In [ ]:
# WGS84 coordinates for clipping
minx, miny = -74.5, 40.6
maxx, maxy = -74.4, 40.7
bbox = box(minx, miny, maxx, maxy) 

In [ ]:
# Insert the box into a GeoDataFrame
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))

In [ ]:
# Re-project into the same coordinate system as the raster data
geo = geo.to_crs(crs=data.crs.data)

In [ ]:
# Convert features into a rasterio-friendly format
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
coords = getFeatures(geo)

In [ ]:
print(coords)

In [ ]:
# Clip raster based on mask data
from rasterio.mask import mask
out_img, out_transform = mask(data, shapes=coords, crop=True)

In [ ]:
# Now need to modify the metadata
out_meta = data.meta.copy()
out_meta

In [ ]:
out_meta.update({"driver": "GTiff",
                 "height": out_img.shape[1],
                 "width": out_img.shape[2],
                 "transform": out_transform})

In [ ]:
with rasterio.open(clip, 'w', **out_meta) as dest:
    dest.write(out_img)

In [ ]:
clip_raster = rasterio.open(clip)
rasterio.plot.show(clip_raster)

## Polygonize the new raster

In [ ]:
gdal.UseExceptions

In [ ]:
# Get raster datsource
sourceRaster = gdal.Open('../data/Created/nj_residential.tif')
sr_proj=sourceRaster.GetProjection()
raster_proj = osr.SpatialReference()
raster_proj.ImportFromWkt(sr_proj)

In [ ]:
band = sourceRaster.GetRasterBand(1) 
bandArray = band.ReadAsArray()

In [ ]:
outShapefile = '../data/Created/nj_residential'

In [ ]:
driver = ogr.GetDriverByName("ESRI Shapefile")
outDatasource = driver.CreateDataSource(outShapefile+ ".shp")
outLayer = outDatasource.CreateLayer('polygonized', srs=raster_proj)
newField = ogr.FieldDefn('LandType', ogr.OFTInteger)
outLayer.CreateField(newField)

gdal.Polygonize( band, None, outLayer, 0, [], callback=None )
outDatasource.Destroy()
sourceRaster = None

In [ ]:
res_geodata = gpd.read_file(outShapefile + '.shp')
res_geodata.head(20)

In [ ]:
# Plot the new file
fig, ax = plt.subplots()
res_geodata.plot(ax=ax,)

plt.show()

In [ ]:
# Identify only residential land
res_only = res_geodata[res_geodata.LandType == 1]
res_only.head(20)

In [ ]:
# Confibrm res_only has less items in the list
len(res_only), len(res_geodata)

In [ ]:
# Aggregate data into one polygon
all_res = res_only.dissolve(by='LandType')

In [ ]:
# Save to a shapefile and GeoJSON
all_res.to_file('../data/Created/all_res.shp', driver='ESRI Shapefile')
all_res.to_file('../data/Created/all_res.json', driver='GeoJSON')

## Prepare the Census and residential data for analysis

In [6]:
# Import NJ public water system data
pws_geodata = gpd.read_file('../data/NJDEP/New_Jersey__Public_Community_Water_Purveyor_Service_Areas.shp')

In [4]:
# Import my census tract/census data shapefile
census_geodata = gpd.read_file('../data/Created/NJ_cen_data.shp')

In [5]:
# Check coordinate system of PWS data
print("PWS coordinate system: {pws}".format(pws=pws_geodata.crs))
print("Residential coordinate system: {res}".format(res=all_res.crs))
print("Census coordinate system: {census}".format(census=census_geodata.crs))

PWS coordinate system: {'init': 'epsg:102711'}


NameError: name 'all_res' is not defined

In [7]:
# Re-project census tract data to NJ public water system projection
census_geodata = census_geodata.to_crs({'init': 'epsg:3424'})
census_geodata.crs

{'init': 'epsg:3424'}

In [ ]:
# Re-project land data to NJ public water system projection
all_res = all_res.to_crs({'init': 'epsg:3424'})
all_res.crs

## Predict the population within the water systems using dasymetric mapping

1. Find the intersection of of residential land and census tracts (referred to below as residential tracts). The result will be a layer representing the primary locations the population resides, and will include the census population data.

In [8]:
# Read saved file as above process is slow
all_res = gpd.read_file('../data/Created/all_res.shp')

In [9]:
# Re-project land data to NJ public water system projection
all_res = all_res.to_crs({'init': 'epsg:3424'})
all_res.crs

{'init': 'epsg:3424'}

In [10]:
all_res

,FID,geometry
0,0,(POLYGON ((361401.3257702506 36185.20584317789...


In [ ]:
# Get intersection of residential area and census tracts
res_tract = gpd.overlay(all_res, census_geodata, how='intersection')

In [ ]:
res_tract['Res_area'] = res_tract.geometry.area

In [ ]:
with pd.option_context('display.max_columns', None):
    display(pd.DataFrame(res_tract.head(5)))

In [ ]:
res_tract.info()

2.  Find the intersection of the new residential tract layer and the PWS system layer and use areal weighting to determine the percentage of the population of each residential tract within the water sytem. 

In [ ]:
# Confirm the coordinate systems match
print(res_tract.crs, pws_geodata.crs)

In [ ]:
intersect = gpd.overlay(res_tract, pws_geodata, how="intersection")

In [ ]:
with pd.option_context('display.max_columns', None):
    display(pd.DataFrame(intersect.head(5)))

In [ ]:
intersect.info()

In [ ]:
# Remove unecessary rows and change some names to be more clear
intersect = intersect.drop(columns=['AREALAND','AREAWATER','state','county','tract_1','CENTLAT','CENTLON','STGEOMETRY', 'STGEOMET_1','PWID_URL', 'AGENCY_URL','USER_LAST_','TMSP_LAST_','SHAPE_Leng'])
intersect = intersect.rename(columns={'SHAPE_Area': 'PWS_Area'})

In [ ]:
intersect.info()

In [ ]:
intersect['Intersect_area'] = intersect.geometry.area

In [ ]:
intersect['Intersect_total_pop'] = (intersect['Intersect_area'] / intersect['Res_area']) * intersect['total_pop']
intersect['Intersect_pov_pop'] = (intersect['Intersect_area'] / intersect['Res_area']) * intersect['pov_pop']
intersect['Intersect_POC_pop'] = (intersect['Intersect_area'] / intersect['Res_area']) * intersect['POC_pop']

In [ ]:
# Plot Intersection
fig, ax = plt.subplots(figsize = (8, 10))
intersect.plot(ax=ax,
                    facecolor='white',
                    edgecolor='grey')


plt.axis('equal')
plt.show()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(intersect.sort_values(by=['GEOID']).head(5))

In [ ]:
# Select the columns I will retain
PWS_dissolve = intersect[['PWID','SYS_NAME','geometry','Intersect_area','Intersect_total_pop','Intersect_pov_pop','Intersect_POC_pop']]

In [ ]:
PWS_dissolve['geometry'] = PWS_dissolve.buffer(0.01)

In [ ]:
# Dissolve and summarize the quantitative columns by sum
PWS_agg = PWS_dissolve.dissolve(by='PWID',aggfunc='sum')

In [ ]:
# Reset the index so the PWID is a column
PWS_agg = PWS_agg.reset_index()

In [ ]:
PWS_agg.info()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(PWS_agg.head(5))

In [ ]:
# Add in columns for percentage people below the poverty line and percentage 
# people of color
PWS_agg['perc_pov'] = PWS_agg['Intersect_pov_pop'] / PWS_agg['Intersect_total_pop']
PWS_agg['perc_POC'] = PWS_agg['Intersect_POC_pop'] / PWS_agg['Intersect_total_pop']

In [ ]:
# Get the NJ state boundary from the TIGER API to plot with the data (see Census
# Jupyter Notebook for more exploration of this process)
con = cen.base.Connection('ACSDT5Y2017')
con.set_mapservice('tigerWMS_ACS2017')

#Get state boundary data
NJ_boundary = con.mapservice.query(layer=82, where='STATE=34',pkg='geopandas')

In [ ]:
# The current way the crs is stored is a dictionary using latestwkid, which causes errors during export. I converted the crs 
# dictionary into one readable by geopandas, but it is not transforming the data in any way. 
NJ_boundary.crs = {'init': 'epsg:3857'}

In [ ]:
#Transform to same projection as PWS data
NJ_boundary=NJ_boundary.to_crs(PWS_agg.crs)

In [ ]:
# Plotting percent below the poverty line by public water system
# Plot the data
fig, ax = plt.subplots(figsize = (8,12))
NJ_boundary.plot(ax=ax,
                edgecolor='black',
                facecolor='white')
PWS_agg.plot(ax = ax,
             column='perc_pov', 
             scheme='QUANTILES',
             k=5,cmap='OrRd',
             edgecolor='black',
             linewidth=0.4, 
             legend=True)

# Geopandas source codes as the legend as a second axis
legend = ax.get_legend()
legend.get_frame().set_linewidth(0.0)
#legend.set(title = 'Percentage Below the Poverty Line')

# Add title to map
ax.set(title = "Percent Below the Poverty Line by Public Water System\n" +
       "New Jersey\n"+
       "New Jersey Department of Environmental Protection Data")

# Turn off the axis
plt.axis('equal')
#ax.set_axis_off()
plt.show()

In [ ]:
# Plotting percent people of color by public water system
# Plot the data
fig, ax = plt.subplots(figsize = (8,12))
NJ_boundary.plot(ax=ax,
                edgecolor='black',
                facecolor='white')
PWS_agg.plot(ax = ax,
             column='perc_POC', 
             scheme='QUANTILES',
             k=5,cmap='OrRd',
             edgecolor='black',
             linewidth=0.4, 
             legend=True)

# Geopandas source codes as the legend as a second axis
legend = ax.get_legend()
legend.get_frame().set_linewidth(0.0)
#legend.set(title = 'Percentage Below the Poverty Line')

# Add title to map
ax.set(title = "Percent People of Color by Public Water System\n" +
       "New Jersey\n"+
       "New Jersey Department of Environmental Protection Data")

# Turn off the axis
plt.axis('equal')
#ax.set_axis_off()

In [ ]:
#Export to shapefile
PWS_agg.to_file("r'C:\Users\zstat\Documents\RecurseCenter\EJ-analysis-map\Data\Created\SpatialAnalysis\Dasymetric.shp", driver='ESRI Shapefile')

In [ ]:
# Export to GeoJSON
# First, need to upcast everything into the "multi-polygon" type

from shapely import geometry
upcast_dispatch = {geometry.Point: geometry.MultiPoint, 
                   geometry.LineString: geometry.MultiLineString, 
                   geometry.Polygon: geometry.MultiPolygon}

def maybe_cast_to_multigeometry(geom):
    caster = upcast_dispatch.get(type(geom), lambda x: x[0])
    return caster([geom])

PWS_agg_json = PWS_agg
PWS_agg_json['geometry'] = PWS_agg_json.geometry.apply(maybe_cast_to_multigeometry)
PWS_agg_json

In [ ]:
# Export to GeoJSON
PWS_agg.to_file("../data/Created/SpatialAnalysis/dasymetric.gojson", driver='GeoJSON')

In [ ]:
geopandas --version